# GPT-5 New Params and Tools

## Imports and Setup

In [29]:
from openai import OpenAI
from IPython.display import display
import pandas as pd
import time


# Display nicely with centered headers
def display_df(df):
    with pd.option_context('display.max_colwidth', None):
        styled_df = df.style.set_table_styles(
            [
                {'selector': 'th', 'props': [('text-align', 'center')]},  # Center column headers
                {'selector': 'td', 'props': [('text-align', 'left')]}     # Left-align table cells
            ]
        )

        display(styled_df)

## Reasoning strength

In [ ]:
client = OpenAI()

question = """
A particle with mass m is in an infinite square well potential with
walls at x = -L/2 and x = L/2.  Write the wave functions for the states
n = 1, n = 2 and n = 3. Answer succinctly.
"""

data = []

for effort in ["minimal", "low", "medium", "high"]:
    print(f"Requesting with effort: {effort}")
    start_time = time.time()
    response = client.responses.create(
        model="gpt-5-nano",
        input=question,
        reasoning={"effort": effort}
    )
    elapsed_time = time.time() - start_time
    print(f"Response: {response}")

    data.append({
        "Effort": effort,
        "Output Text": response.output_text,
        "Reasoning Tokens": response.usage.output_tokens_details.reasoning_tokens,
        "Output Tokens": response.usage.output_tokens,
        "Time (s)": round(elapsed_time, 2)
    })

# Create DataFrame
df = pd.DataFrame(data)

Requesting with effort: minimal
Response: Response(id='resp_68975ffebbe481a29e26b947a30d9b0b0700620b456d1f08', created_at=1754750974.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-nano-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_68975fff01d881a29c4ed75d23086b4c0700620b456d1f08', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseOutputMessage(id='msg_68975fff0e4881a2840311489a7d29f40700620b456d1f08', content=[ResponseOutputText(annotations=[], text='For a particle in an infinite square well of width L with walls at x = -L/2 and x = L/2, the normalized stationary states are:\n\n- n = 1: psi1(x) = sqrt(2/L) cos(pi x / L)\n- n = 2: psi2(x) = sqrt(2/L) sin(2 pi x / L)\n- n = 3: psi3(x) = sqrt(2/L) cos(3 pi x / L)\n\n(Note: parity alternating: odd n -> cosine, even n -> sine, with the given symmetric well.)', type='output_text', logprobs=[])], role='assistant', status='completed', type='mes

In [16]:
display_df(df)

,Effort,Sample Output,Reasoning Tokens
0,minimal,"For a particle in an infinite square well of width L with walls at x = -L/2 and x = L/2, the normalized stationary states are: - n = 1: psi1(x) = sqrt(2/L) cos(pi x / L) - n = 2: psi2(x) = sqrt(2/L) sin(2 pi x / L) - n = 3: psi3(x) = sqrt(2/L) cos(3 pi x / L) (Note: parity alternating: odd n -> cosine, even n -> sine, with the given symmetric well.)",0
1,high,"For a well of width L from -L/2 to L/2, the first three normalized eigenfunctions are: - ψ1(x) = sqrt(2/L) cos(π x / L) - ψ2(x) = sqrt(2/L) sin(2π x / L) - ψ3(x) = sqrt(2/L) cos(3π x / L) (Outside |x| ≥ L/2: ψn(x) = 0.)",7104


## Verbosity

In [ ]:
client = OpenAI()

question = "Explain the benefits of cooking with a cast iron pan"

data = []

for verbosity in ["low", "medium", "high"]:
    print(f"Requesting with verbosity: {verbosity}")
    response = client.responses.create(
        model="gpt-5-nano",
        input=question,
        text={"verbosity": verbosity}
    )
    print(f"Response: {response}")

    data.append({
        "Verbosity": verbosity,
        "Output Text": response.output_text,
        "Output Tokens": response.usage.output_tokens
    })

# Create DataFrame
df = pd.DataFrame(data)

Requesting with verbosity: low
Response: Response(id='resp_68975db93ee081a2b432330818241af80a0267a32387dd28', created_at=1754750393.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-nano-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_68975db9e80481a2b17a0183266e85c50a0267a32387dd28', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseOutputMessage(id='msg_68975dbdec9481a28d6e436ceee8599a0a0267a32387dd28', content=[ResponseOutputText(annotations=[], text='Cast iron pans offer several practical benefits:\n\n- Superior heat retention and even heating, great for searing, frying, and browning.\n- Extremely versatile: stovetop, oven, grill, or campfire; can go from heat to heat without warping.\n- Develops a durable, natural nonstick coating (seasoning) with use; improves with time.\n- Creates fond for easy pan sauces and flavorful results.\n- Very durable and often inexpensive; can last for gener

In [9]:
display_df(df)

## Freeform Function Calling

In [12]:
client = OpenAI()

response = client.responses.create(
    model="gpt-5-nano",
    input="Use the exec_bash_command tool to list the files in the /Users/alex/pro/zazencodes-season-2/src directory using eza (tree depth 2)",
    text={"format": {"type": "text"}},
    tools=[
        {
            "type": "custom",
            "name": "exec_bash_command",
            "description": "Executes arbitrary bash commands",
        }
    ]
)

In [13]:
response.output

[ResponseReasoningItem(id='rs_689766c10dac8192b6fcd2d04d6e7a230f1fc3ee71babab9', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseCustomToolCall(call_id='call_zkKTm3MS04m04Z9rQKoumfAJ', input='eza --tree -L 2 /Users/alex/pro/zazencodes-season-2/src', name='exec_bash_command', type='custom_tool_call', id='ctc_689766c521688192bb7ae2691e71ee5b0f1fc3ee71babab9', status='completed')]

In [14]:
response.output[-1].to_dict()

{'call_id': 'call_zkKTm3MS04m04Z9rQKoumfAJ',
 'input': 'eza --tree -L 2 /Users/alex/pro/zazencodes-season-2/src',
 'name': 'exec_bash_command',
 'type': 'custom_tool_call',
 'id': 'ctc_689766c521688192bb7ae2691e71ee5b0f1fc3ee71babab9',
 'status': 'completed'}

In [15]:
import subprocess

def bash_command_tool(command: str) -> str:
    print(subprocess.run(command, shell=True, capture_output=True, text=True).stdout)
    return 

In [16]:
"""
This cell will execute arbitrary bash commands.
USE WITH CAUTION!
"""

for item in response.output:
    if item.type == "custom_tool_call" and item.name == "exec_bash_command":
        bash_command_tool(item.input)
        

/Users/alex/pro/zazencodes-season-2/src
├── ai-eng-tier-lists
│   └── startups
├── ai-scientific-research-agent
│   ├── dev
│   ├── DEVLOG.md
│   ├── notebooks
│   ├── output
│   ├── README.md
│   ├── requirements.txt
│   ├── sci_research_agent
│   └── setup.py
├── anatomy-of-a-system-prompt
│   ├── anatomy_of_a_system_prompt.html
│   ├── anatomy_of_a_system_prompt.md
│   ├── anatomy_of_a_system_prompt.pdf
│   ├── chad_gpt_system_prompt.md
│   ├── chad_gpt_user_questions.md
│   ├── dracula.css
│   ├── images
│   ├── README.md
│   └── render_slides.sh
├── claude-code-sub-agents
│   ├── anime-quote-fetcher
│   ├── dracula-style-plots
│   └── plant-helper-mcp
├── data-jobs-2025
│   ├── job_tables.md
│   ├── jobs_hierarchy.md
│   └── README.md
├── deepseek-ollama
│   ├── favorites.json
│   ├── README.md
│   └── test.py
├── discord-ai-bot-trivia-greetings
│   ├── Dockerfile
│   ├── main.py
│   ├── README.md
│   ├── requirements.txt
│   ├── test_add_user.py
│   └── test_send_message.py
├── d

## Context‑Free Grammar (CFG)

CFG's are effectively contracts that force the model to emit only strings that the grammar accept

In [89]:
client = OpenAI()

# + followed by 8–15 digits total, first digit 1–9 (E.164). Add |UNKNOWN to allow a fallback.
phone_grammar_definition = r"^(?:\+[1-9]\d{7,14}|UNKNOWN)$"

def normalize_phone(prompt: str) -> OpenAI:
    resp = client.responses.create(
        model="gpt-5-nano",
        input=prompt,
        tools=[{
            "type": "custom",
            "name": "e164_phone",
            "description": "Return E.164 phone or UNKNOWN.",
            "format": {
                "type": "grammar",
                "syntax": "regex",
                "definition": phone_grammar_definition
            }
        }],
    )
    return resp


resp = normalize_phone("Convert the phone number (123) 456-7890 to E.164 or output UNKNOWN")

In [90]:
resp

Response(id='resp_689797919a0c819484170b0a91b214b80b24c035c104fa39', created_at=1754765204.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-nano-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_68979794cdc48194b271f9d0322cc2d60b24c035c104fa39', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseCustomToolCall(call_id='call_hkSjvBlExZotaMcOwIiz3EOW', input='+11234567890', name='e164_phone', type='custom_tool_call', id='ctc_6897979996488194ad0fa6991e9a30820b24c035c104fa39', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[CustomTool(name='e164_phone', type='custom', description='Return E.164 phone or UNKNOWN.', format=Grammar(definition='^(?:\\+[1-9]\\d{7,14}|UNKNOWN)$', syntax='regex', type='grammar'))], top_p=1.0, background=False, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Re

In [91]:
resp.output_text

''

In [92]:
resp.output

[ResponseReasoningItem(id='rs_68979794cdc48194b271f9d0322cc2d60b24c035c104fa39', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseCustomToolCall(call_id='call_hkSjvBlExZotaMcOwIiz3EOW', input='+11234567890', name='e164_phone', type='custom_tool_call', id='ctc_6897979996488194ad0fa6991e9a30820b24c035c104fa39', status='completed')]

In [93]:
resp.output[-1].to_dict()

{'call_id': 'call_hkSjvBlExZotaMcOwIiz3EOW',
 'input': '+11234567890',
 'name': 'e164_phone',
 'type': 'custom_tool_call',
 'id': 'ctc_6897979996488194ad0fa6991e9a30820b24c035c104fa39',
 'status': 'completed'}

In [61]:
resp2 = normalize_phone("Convert the Canadian phone number 456-780 to E.164 or output UNKNOWN")

In [62]:
resp2

Response(id='resp_689793befb5c819fb3776022700d69b10f9c98a638d89ad3', created_at=1754764226.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-nano-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_689793c29844819fa7cdc11eece500e30f9c98a638d89ad3', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseCustomToolCall(call_id='call_M0DLo2eqskOjp3RxUGV1RtYm', input='UNKNOWN', name='e164_phone', type='custom_tool_call', id='ctc_689793c4e920819fb8367e2bc23773ff0f9c98a638d89ad3', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[CustomTool(name='e164_phone', type='custom', description='Return E.164 phone or UNKNOWN.', format=Grammar(definition='^(?:\\+[1-9]\\d{7,14}|UNKNOWN)$', syntax='regex', type='grammar'))], top_p=1.0, background=False, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoni

In [63]:
resp2.output_text

''

In [64]:
resp2.output

[ResponseReasoningItem(id='rs_689793c29844819fa7cdc11eece500e30f9c98a638d89ad3', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseCustomToolCall(call_id='call_M0DLo2eqskOjp3RxUGV1RtYm', input='UNKNOWN', name='e164_phone', type='custom_tool_call', id='ctc_689793c4e920819fb8367e2bc23773ff0f9c98a638d89ad3', status='completed')]

In [65]:
resp2.output[-1].to_dict()

{'call_id': 'call_M0DLo2eqskOjp3RxUGV1RtYm',
 'input': 'UNKNOWN',
 'name': 'e164_phone',
 'type': 'custom_tool_call',
 'id': 'ctc_689793c4e920819fb8367e2bc23773ff0f9c98a638d89ad3',
 'status': 'completed'}

In [109]:
resp3 = normalize_phone("Output the iconic first sentence of the book Neuromancer")

In [110]:
resp3.output_text

'The sky above the port was the color of television, tuned to a dead channel.'

In [69]:
resp3.output

[ResponseReasoningItem(id='rs_689793f1b958819f9ddcb19b4726291a0a1a49e4cf2c8159', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseOutputMessage(id='msg_689793fac9fc819fa163693415d739700a1a49e4cf2c8159', content=[ResponseOutputText(annotations=[], text='The sky above the port was the color of television,', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]

In [70]:
resp3.output[-1].to_dict()

{'id': 'msg_689793fac9fc819fa163693415d739700a1a49e4cf2c8159',
 'content': [{'annotations': [],
   'text': 'The sky above the port was the color of television,',
   'type': 'output_text',
   'logprobs': []}],
 'role': 'assistant',
 'status': 'completed',
 'type': 'message'}

In [115]:
client = OpenAI()

# Accepts common single-codepoint emojis and their sequences (ZWJ, VS-16, skin tones, flags, keycaps).
# If you want to allow spaces/newlines between emojis, add \s to the character class.
emoji_grammar_definition = (
    r"^(?:"
    r"[\u2600-\u26FF]"                # Misc Symbols
    r"|[\u2700-\u27BF]"               # Dingbats
    r"|[\U0001F1E6-\U0001F1FF]"       # Regional indicators (flags)
    r"|[\U0001F300-\U0001F5FF]"       # Misc Symbols & Pictographs
    r"|[\U0001F600-\U0001F64F]"       # Emoticons
    r"|[\U0001F680-\U0001F6FF]"       # Transport & Map
    r"|[\U0001F700-\U0001F77F]"       # Alchemical Symbols
    r"|[\U0001F780-\U0001F7FF]"       # Geometric Shapes Extended
    r"|[\U0001F900-\U0001F9FF]"       # Supplemental Symbols & Pictographs
    r"|[\U0001FA70-\U0001FAFF]"       # Symbols & Pictographs Extended-A
    r"|[\U0001F3FB-\U0001F3FF]"       # Emoji skin tone modifiers
    r"|\u200D"                         # Zero-width joiner (ZWJ)
    r"|\uFE0F"                         # Variation Selector-16
    r"|\u20E3"                         # Combining enclosing keycap
    r")+$"
)

def ask_emoji(prompt: str):
    resp = client.responses.create(
        model="gpt-5-mini",
        input=prompt,
        text={"verbosity": "high"},
        tools=[{
            "type": "custom",
            "name": "emoji_grammar",
            "description": "Outputs a string containing only emojis. You must ensure the text contains only emojis and no other characters.",
            "format": {
                "type": "grammar",
                "syntax": "regex",
                "definition": emoji_grammar_definition
            }
        }],
    )

    return resp

# Example:
resp = ask_emoji("Summarize quantum computing using just emojis and check your work with the emoji_grammar tool")


In [116]:
resp.output_text

''

In [117]:
resp.output

[ResponseReasoningItem(id='rs_68979b24bc0481a19475c27e55fe25010943a5a8475e43ad', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseCustomToolCall(call_id='call_Uk2vs1i3gssKwN9pe9QESobp', input='⚛️🔀🔗⚙️🌊🔬➡️🎲⚡📈🧠💻🧊🛠️🚀', name='emoji_grammar', type='custom_tool_call', id='ctc_68979b40f2d081a19d4797e47e9059480943a5a8475e43ad', status='completed')]

In [118]:
resp.output[-1].to_dict()

{'call_id': 'call_Uk2vs1i3gssKwN9pe9QESobp',
 'input': '⚛️🔀🔗⚙️🌊🔬➡️🎲⚡📈🧠💻🧊🛠️🚀',
 'name': 'emoji_grammar',
 'type': 'custom_tool_call',
 'id': 'ctc_68979b40f2d081a19d4797e47e9059480943a5a8475e43ad',
 'status': 'completed'}